# IMPORTING LIBRARIES

In [21]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

## ACCESSING DATA VIA API

In [22]:
conn = http.client.HTTPSConnection("api.covid19api.com")
payload = ''
headers = {}
conn.request("GET","/summary",payload,headers)
res = conn.getresponse()
data = res.read().decode('UTF-8')

# CONVERT DATA TO JSON

In [23]:
covid1 = json.loads(data)

## NORMALIZE DATA

In [24]:
pd.json_normalize(covid1['Countries'],sep=",")

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,19,38162,0,1402,26,29089,2020-08-31T06:36:01Z
1,Albania,AL,albania,101,9380,5,280,119,5139,2020-08-31T06:36:01Z
2,Algeria,DZ,algeria,365,44146,10,1501,261,30978,2020-08-31T06:36:01Z
3,Andorra,AD,andorra,0,1124,0,53,0,902,2020-08-31T06:36:01Z
4,Angola,AO,angola,73,2624,0,107,22,1063,2020-08-31T06:36:01Z
...,...,...,...,...,...,...,...,...,...,...
181,Viet Nam,VN,vietnam,0,1040,0,32,18,695,2020-08-31T06:36:01Z
182,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-08-31T06:36:01Z
183,Yemen,YE,yemen,7,1953,1,564,10,1123,2020-08-31T06:36:01Z
184,Zambia,ZM,zambia,123,12025,3,287,362,11454,2020-08-31T06:36:01Z


## CONVERTING TO PANDAS DATAFRAME

In [25]:
df = pd.DataFrame(covid1['Countries'])
df

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date,Premium
0,Afghanistan,AF,afghanistan,19,38162,0,1402,26,29089,2020-08-31T06:36:01Z,{}
1,Albania,AL,albania,101,9380,5,280,119,5139,2020-08-31T06:36:01Z,{}
2,Algeria,DZ,algeria,365,44146,10,1501,261,30978,2020-08-31T06:36:01Z,{}
3,Andorra,AD,andorra,0,1124,0,53,0,902,2020-08-31T06:36:01Z,{}
4,Angola,AO,angola,73,2624,0,107,22,1063,2020-08-31T06:36:01Z,{}
...,...,...,...,...,...,...,...,...,...,...,...
181,Viet Nam,VN,vietnam,0,1040,0,32,18,695,2020-08-31T06:36:01Z,{}
182,Western Sahara,EH,western-sahara,0,10,0,1,0,8,2020-08-31T06:36:01Z,{}
183,Yemen,YE,yemen,7,1953,1,564,10,1123,2020-08-31T06:36:01Z,{}
184,Zambia,ZM,zambia,123,12025,3,287,362,11454,2020-08-31T06:36:01Z,{}


## <center> DROPPING UNNECESSARY COLUMNS <center>

In [26]:
covid2 = df.drop(columns=['CountryCode','Slug','Date','Premium'])
covid2

,Country,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,Afghanistan,19,38162,0,1402,26,29089
1,Albania,101,9380,5,280,119,5139
2,Algeria,365,44146,10,1501,261,30978
3,Andorra,0,1124,0,53,0,902
4,Angola,73,2624,0,107,22,1063
...,...,...,...,...,...,...,...
181,Viet Nam,0,1040,0,32,18,695
182,Western Sahara,0,10,0,1,0,8
183,Yemen,7,1953,1,564,10,1123
184,Zambia,123,12025,3,287,362,11454


## GENERATING BASE MAPS

In [27]:
m = folium.Map(tiles="Stamen Terrain", min_zoom = 1.5)
m

## OBTANING GEODATA

In [19]:
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [28]:
folium.Choropleth(
    geo_data = country_shapes,
    min_zoom = 2,
    name = 'Covid-19',
    data = covid2,
    columns = ['Country','TotalConfirmed'],
    key_on = 'feature.properties.name',
    fill_color = 'OrRd',
    nan_fill_color = 'black',
    legend_name = "Total Confirmed Covid Cases"
).add_to(m)

m


## Generate Circular Markers

In [29]:
covid2.update(covid2['TotalConfirmed'].map('Total confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total recovered:{}'.format))

In [33]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates

,latitude,longitude,Country
0,33.939110,67.709953,Afghanistan
1,41.153332,20.168331,Albania
2,28.033886,1.659626,Algeria
3,-14.270972,-170.132217,American Samoa
4,42.546245,1.601554,Andorra
...,...,...,...
239,-13.768752,-177.156097,Wallis and Futuna
240,24.215527,-12.885834,Western Sahara
241,15.552727,48.516388,Yemen
242,-13.133897,27.849332,Zambia


In [34]:
covid_final=pd.merge(covid2,coordinates,on='Country')

In [41]:
def plotDot(point):
    folium.CircleMarker(location = [point.latitude,point.longitude],
                       radius = 5,
                       weight = 2,
                       popup = [point.Country,point.TotalConfirmed,point.TotalRecovered],
                       fill_color = '#000000').add_to(m)

In [42]:
covid_final.apply(plotDot,axis = 1)
m.fit_bounds(m.get_bounds())
m

## GENERATE BASE MAP

In [45]:
m1 = folium.Map(tiles='StamenToner',min_zoom=2)

In [46]:
deaths = covid_final['TotalDeaths'].astype(float)

lat = covid_final['latitude'].astype(float)

lon = covid_final['longitude'].astype(float)

## GENERATING HEAT MAP

In [47]:
m1.add_child(HeatMap(zip(lat,lon,deaths),radius=0))

## GENERATING CIRCULAR MARKS

In [53]:
def plotDot(point):
    folium.CircleMarker(location = [point.latitude,point.longitude],
                       radius = 5,
                       weight = 2,
                       popup = [point.Country,point.TotalConfirmed,point.TotalRecovered],
                       fill_color = '#000000').add_to(m1)

In [54]:
covid_final.apply(plotDot,axis = 1)
m1.fit_bounds(m.get_bounds())
m1